## The Protein Puzzle: Unraveling the Complexities of Diet and Life Expectancy

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
sns.set_style("whitegrid")
from importlib import reload
plt=reload(plt)


In [ ]:
df.columns


# cleaning

In [ ]:
protein_one = pd.read_csv('/Users/jonathanfarro/Desktop/animal-protein-consumption.csv')

life_ex = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/life-expectancy-hmd-unwpp.csv')

usa_pricing = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/average_prices_for_food.csv')

daily_cals = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/daily-caloric-supply-derived-from-carbohydrates-protein-and-fat.csv')

fruit_per_cap = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/fruit-consumption-per-capita.csv')

veg_per_cap = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/vegetable-consumption-per-capita/vegetable-consumption-per-capita.csv' )

nutrition =  pd.read_excel('/Users/jonathanfarro/Desktop/ Capstone Files/food_comp.xlsx')

protein_per_GDP = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/daily-per-capita-protein-supply-vs-gdp-per-capita.csv')

death = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/death-rate-by-cause-who-mdb.csv')



In [ ]:
# rename columns to something easier to work with 

protein = protein_one.rename(columns={
                        protein_one.columns[3]:'fish',
                        protein_one.columns[4]:'poultry',
                        protein_one.columns[5]:'pork',
                        protein_one.columns[6]:'beef',
                        protein_one.columns[7]:'sheep/goat',
                        protein_one.columns[8]:'other',
                        protein_one.columns[9]:'eggs',
                        protein_one.columns[10]:'milk'
                       })

In [ ]:
# bulk clean up 
protein = protein.rename(columns=lambda x: x.lower())
protein = protein.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)

life_ex = life_ex.rename(columns = lambda x: x.lower())
life_ex = life_ex.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)
life_ex = life_ex.rename(columns = {life_ex.columns[3] : 'life_expectancy(0)'})

protein_per_GDP = protein_per_GDP.rename(columns = lambda x: x.lower())
protein_per_GDP = protein_per_GDP.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)
gdp_population = protein_per_GDP.drop(columns= {protein_per_GDP.columns[3],protein_per_GDP.columns[6]})
gdp_population = gdp_population.rename(columns = { gdp_population.columns[3]: 'gdp_per_capita'}) 


daily_cals = daily_cals.rename(columns = lambda x: x.lower())
daily_cals = daily_cals.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

death = death.rename(columns = lambda x: x.lower())
death = death.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

nutrition = nutrition.rename(columns = lambda x: x.lower())
nutrition = nutrition.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)

fruit_per_cap = fruit_per_cap.rename(columns=lambda x: x.lower())
fruit_per_cap = fruit_per_cap.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)
fruit = fruit_per_cap.rename(columns = {fruit_per_cap.columns[3]:'fruit_kilo_yearly'}).reset_index(drop = True)

# creating a new column to show total_protein intake for a day.
protein['total_protein'] = protein.iloc[:, 3:11].sum(axis = 1).round(2)



In [ ]:
capstone = pd.merge(protein,
         life_ex,
         on= ['entity','code','year'],
         how= 'inner')

#capstone.head()

capstone_2 = pd.merge( capstone, gdp_population, on = ['entity','code','year'],how= 'inner')

main_frame = pd.merge(capstone_2,death,  on = ['entity','code','year'],how= 'outer')

final_frame = pd.merge(main_frame, daily_cals,  on = ['entity','code','year'],how= 'outer')

final_frame['total_daily_protein']=final_frame.iloc[:, 3:10].sum(axis=1)

final_frame = final_frame.rename(columns= { 'daily calorie supply per person that comes from animal protein':'daily_cal_animal_protein',
                                            'daily calorie supply per person that comes from vegetal protein':'daily_cal_veg_protein',
                                            'daily calorie supply per person from fat':'daily_cal_fat',
                                            'daily calorie supply per person from carbohydrates':'daily_cal_carbs'})
final_frame.entity.unique()




In [ ]:
#creating an all_cause_mortality column, and more cleaning

final_frame['all_cause_per_100k'] = final_frame.iloc[:,15:24].sum(axis=1)

final_frame_two = final_frame.drop(columns=final_frame.columns[15:24])

final_frame_three = final_frame_two[~final_frame_two['entity'].str.contains(r'\(.*\)')]

final = final_frame_three
final = final.rename(columns={'life_expectancy(0)':'life_exp','population (historical)':'population'})

In [ ]:
#further cleaning.

non_countries_list = ['asia','europe','high-income countries','low-income countries','lower-middle-income countries','oceania','upper-middle-income countries','world']
income_classes= ['low-income countries','high-income countries','lower-middle-income countries','upper-middle-income countries','world']

final_countries = final.loc[~final['entity'].isin(non_countries_list)]

income_class = final.loc[final['entity'].isin(income_classes)]

income_class['veg_protein_ratio'] = income_class['daily_cal_veg_protein']/ income_class['daily_cal_animal_protein']
income_class['animal_protein_ratio']= income_class['daily_cal_animal_protein']/income_class['daily_cal_veg_protein']




testdrop= final_countries

df = testdrop.dropna()

df['cals_from_protein'] = df['daily_cal_animal_protein'] + df['daily_cal_veg_protein']

df['daily_caloric_intake'] = df.iloc[:,15:19].sum(axis=1)
df['non_protein_cals']= df['daily_cal_fat'] + df['daily_cal_carbs'] 
income_class['daily_caloric_intake'] = income_class.iloc[:,15:19].sum(axis=1)


df['veg_protein_ratio'] = df['daily_cal_veg_protein']/  df['daily_cal_animal_protein']
df['animal_protein_ratio']= df['daily_cal_animal_protein']/df['daily_cal_veg_protein']



In [ ]:
income_class = income_class.loc[income_class['entity']!= 'world']


In [ ]:
country_to_region = {
    'algeria': 'Africa',
    'angola': 'Africa',
    'benin': 'Africa',
    'botswana': 'Africa',
    'burkina faso': 'Africa',
    'burundi': 'Africa',
    'cabo verde': 'Africa',
    'cameroon': 'Africa',
    'central african republic': 'Africa',
    'chad': 'Africa',
    'comoros': 'Africa',
    'congo, democratic republic of the': 'Africa',
    'congo, republic of the': 'Africa',
    "côte d'ivoire": 'Africa',
    'djibouti': 'Africa',
    'egypt': 'Africa',
    'equatorial guinea': 'Africa',
    'eritrea': 'Africa',
    'eswatini': 'Africa',
    'ethiopia': 'Africa',
    'gabon': 'Africa',
    'gambia': 'Africa',
    'ghana': 'Africa',
    'guinea': 'Africa',
    'guinea-bissau': 'Africa',
    'kenya': 'Africa',
    'lesotho': 'Africa',
    'liberia': 'Africa',
    'libya': 'Africa',
    'madagascar': 'Africa',
    'malawi': 'Africa',
    'mali': 'Africa',
    'mauritania': 'Africa',
    'mauritius': 'Africa',
    'morocco': 'Africa',
    'mozambique': 'Africa',
    'namibia': 'Africa',
    'niger': 'Africa',
    'nigeria': 'Africa',
    'rwanda': 'Africa',
    'sao tome and principe': 'Africa',
    'senegal': 'Africa',
    'seychelles': 'Africa',
    'sierra leone': 'Africa',
    'somalia': 'Africa',
    'south africa': 'Africa',
    'south sudan': 'Africa',
    'sudan': 'Africa',
    'tanzania': 'Africa',
    'togo': 'Africa',
    'tunisia': 'Africa',
    'uganda': 'Africa',
    'zambia': 'Africa',
    'zimbabwe': 'Africa',
    'afghanistan': 'Asia',
    'armenia': 'Asia',
    'azerbaijan': 'Asia',
    'bahrain': 'Asia',
    'bangladesh': 'Asia',
    'bhutan': 'Asia',
    'brunei': 'Asia',
    'cambodia': 'Asia',
    'china': 'Asia',
    'cyprus': 'Asia', 
    'georgia': 'Asia',
    'india': 'Asia',
    'indonesia': 'Asia',
    'iran': 'Asia',
    'iraq': 'Asia',
    'israel': 'Asia',
    'japan': 'Asia',
    'jordan': 'Asia',
    'kazakhstan': 'Asia',
    'kuwait': 'Asia',
    'kyrgyzstan': 'Asia',
    'laos': 'Asia',
    'lebanon': 'Asia',
    'malaysia': 'Asia',
    'maldives': 'Asia',
    'mongolia': 'Asia',
    'myanmar': 'Asia',
    'nepal': 'Asia',
    'north korea': 'Asia',
    'oman': 'Asia',
    'pakistan': 'Asia',
    'palestine': 'Asia',
    'philippines': 'Asia',
    'qatar': 'Asia',
    'russia': 'Asia', # Mostly in Asia
    'saudi arabia': 'Asia',
    'singapore': 'Asia',
    'south korea': 'Asia',
    'sri lanka': 'Asia',
    'syria': 'Asia',
    'tajikistan': 'Asia',
    'thailand': 'Asia',
    'timor-leste': 'Asia',
    'turkey': 'Asia', # Mostly in Asia
    'turkmenistan': 'Asia',
    'united arab emirates': 'Asia',
    'uzbekistan': 'Asia',
    'vietnam': 'Asia',
    'yemen': 'Asia',
    'macao':'Asia',
    'albania': 'Europe',
    'andorra': 'Europe',
    'austria': 'Europe',
    'belarus': 'Europe',
    'belgium': 'Europe',
    'bosnia and herzegovina': 'Europe',
    'bulgaria': 'Europe',
    'croatia': 'Europe',
    'czech republic': 'Europe',
    'denmark': 'Europe',
    'estonia': 'Europe',
    'finland': 'Europe',
    'france': 'Europe',
    'germany': 'Europe',
    'greece': 'Europe',
    'hungary': 'Europe',
    'iceland': 'Europe',
    'ireland': 'Europe',
    'italy': 'Europe',
    'kosovo': 'Europe',
    'latvia': 'Europe',
    'liechtenstein': 'Europe',
    'lithuania': 'Europe',
    'luxembourg': 'Europe',
    'malta': 'Europe',
    'moldova': 'Europe',
    'monaco': 'Europe',
    'montenegro': 'Europe',
    'netherlands': 'Europe',
    'north macedonia': 'Europe',
    'norway': 'Europe',
    'poland': 'Europe',
    'portugal': 'Europe',
    'romania': 'Europe',
    'russia': 'Europe', 
    'san marino': 'Europe',
    'serbia': 'Europe',
    'slovakia': 'Europe',
    'slovenia': 'Europe',
    'spain': 'Europe',
    'sweden': 'Europe',
    'switzerland': 'Europe',
    'ukraine': 'Europe',
    'united kingdom': 'Europe',
    'vatican city': 'Europe', 
    'antigua and barbuda': 'North America',
    'bahamas': 'North America',
    'barbados': 'North America',
    'belize': 'North America',
    'canada': 'North America',
    'costa rica': 'North America',
    'cuba': 'North America',
    'dominica': 'North America',
    'dominican republic': 'North America',
    'el salvador': 'North America',
    'grenada': 'North America',
    'guatemala': 'North America',
    'haiti': 'North America',
    'honduras': 'North America',
    'jamaica': 'North America',
    'mexico': 'North America',
    'nicaragua': 'North America',
    'panama': 'North America',
    'saint kitts and nevis': 'North America',
    'saint lucia': 'North America',
    'saint vincent and the grenadines': 'North America',
    'trinidad and tobago': 'North America',
    'united states': 'North America',
    'australia': 'Oceania',
    'fiji': 'Oceania',
    'kiribati': 'Oceania',
    'marshall islands': 'Oceania',
    'micronesia': 'Oceania',
    'nauru': 'Oceania',
    'new zealand': 'Oceania',
    'palau': 'Oceania',
    'papua new guinea': 'Oceania',
    'samoa': 'Oceania',
    'solomon islands': 'Oceania',
    'tonga': 'Oceania',
    'tuvalu': 'Oceania',
    'vanuatu': 'Oceania',
    'argentina': 'South America',
    'bolivia': 'South America',
    'brazil': 'South America',
    'chile': 'South America',
    'colombia': 'South America',
    'ecuador': 'South America',
    'guyana': 'South America',
    'paraguay': 'South America',
    'peru': 'South America',
    'suriname': 'South America',
    'uruguay': 'South America',
    'venezuela': 'South America',
}

df['region'] = df['entity'].map(country_to_region)

df['entity'] = df['entity'].str.title()

In [ ]:
df.info()

# correlation chart

In [ ]:
cols_to_include = df.columns[11:15].tolist() + df.columns[15:16].tolist() + df.columns[24:26].tolist() 

# Calculate the correlation matrix
corr = df[cols_to_include].corr()
corr

rs = np.random.RandomState(0)

corr.style.background_gradient(cmap='copper_r')
plt.figure(figsize=(5, 4))  # Adjust figure size as needed
sns.heatmap(corr, annot=True, cmap='copper', fmt=".2f")
plt.title('Correlation Matrix')



#  What is the correlation between the consumption of food types per capita and life expectancy across different countries, and how has this correlation changed over time? 


In [ ]:
df.life_exp.mean().round()

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(10, 5))

sns.set_style("whitegrid", {"grid.color": ".6", "grid.linestyle": ":"})
sns.scatterplot(
    data=df, y="total_daily_protein", x="life_exp",
     marker=(8,2,0),hue='region',palette='copper_r',s=80)

plt.tight_layout()
plt.xlabel("Total Daily Protein Consumption Per Capita")
plt.ylabel('Life Expectancy')
plt.savefig('yolo.png',transparent=True)


In [ ]:
forplot = df.groupby(['region','year']).agg({'life_exp':'mean','total_daily_protein':'mean','gdp_per_capita':'mean'})
forplot = forplot.reset_index()
forplot

In [ ]:
df.life_exp.mean().round()
print(f'The average life expectancy covered in this dataset is {df.life_exp.mean().round()} years old,')
print(f'We can see from the scatterplot above that life expectancy of a country and total protein consumption per day has a strong positive correlation.')

# Do countries with a higher ratio vegetables protein consumption compared to animal protein exhibit higher life expectancies, and how does this relationship vary across different regions?

In [ ]:
# splitting the df to show two styles of diets 
more_meat = df.loc[df['animal_protein_ratio'] > df['veg_protein_ratio']]

more_veg = df.loc[df['veg_protein_ratio'] > df['animal_protein_ratio']]

In [ ]:
plt.figure(figsize=(13, 6))

sns.set_style("whitegrid")
plt.subplot(1, 2, 1)
sns.scatterplot(x= df['veg_protein_ratio'],y=df['life_exp'],hue=df['region'],palette='copper_r')
plt.xlabel('Vegetable Protein Consumption')
plt.legend(loc="lower right")
plt.ylabel('Life Expectancy')
plt.subplot(1, 2, 2)
sns.scatterplot(x= df['animal_protein_ratio'],y=df['life_exp'],hue=df['region'],palette='copper_r')
plt.xlabel('Animal Protein Consumption')
plt.ylabel('Life Expectancy')
plt.legend(loc="best")
plt.savefig('FINALmeatveg.png', transparent=True)

In [ ]:
# seems animal protein is an indicator of a more steady incline of life exp but there are still low outliers and what could be causing that?


# Does this correlation differ between high-income and low-income countries?

In [ ]:
from importlib import reload
plt=reload(plt)

In [ ]:
income_class.columns

In [ ]:
plt.figure(figsize=(13, 5))

plt.subplot(1, 2, 1)
sns.scatterplot(
                income_class.sort_values(by='gdp_per_capita',ascending=False),
                x='daily_cal_animal_protein',
                y='life_exp',hue='entity',
                palette='copper_r',marker="o",s=150,alpha=.5,legend=False)
plt.xlabel('Animal Protein Consumption in Calories Per Capita')
plt.ylabel('Life Expectancy')

plt.subplot(1, 2, 2)
sns.scatterplot(
                income_class.sort_values(by='gdp_per_capita',ascending=False),
                y='life_exp',
                x='daily_cal_veg_protein',hue='entity',
                palette='copper_r',marker="o",s=150,alpha=.5)
plt.xlabel('Vegetable Protein Consumption in Calories Per Capita')
plt.ylabel('Life Expectancy')


plt.legend(loc='best')
plt.ylabel=('')
plt.savefig('incomecountries.png', transparent=True)
;

High-Income Countries:
Access and Affordability: In these countries, access to a diverse range of foods is generally higher due to well-developed food systems and greater affordability. However, lifestyle choices and the prevalence of processed foods can still lead to less-than-optimal diets.

Concerns: The focus in high-income countries is often shifting towards the quality of food and the overconsumption of certain food groups (like processed foods, red meats, sugary drinks), which can contribute to chronic diseases and negatively impact life expectancy.

Low-Income Countries:
Limited Access: Food insecurity and limited access to diverse foods remain major challenges. Diets might be heavily reliant on staple foods, leading to deficiencies in essential nutrients.   
Economic Factors: Economic constraints often dictate food choices, making it difficult for people to prioritize dietary diversity.   
Infectious Diseases: While nutrition plays a crucial role, life expectancy in lbow-income countries is also significantly affected by infectious diseases, access to healthcare, and other socioeconomic factors.

In [ ]:
plt.figure(figsize=(13, 5))

plt.subplot(1, 2, 1)
sns.scatterplot(
                income_class.sort_values(by='gdp_per_capita',ascending=False),
                x='gdp_per_capita',
                y='life_exp',hue='entity',
                palette='copper_r',marker="o",s=150,alpha=.5,legend=False)
plt.xlabel('GDP Per Capita')
plt.ylabel('Life Expectancy')

plt.subplot(1, 2, 2)
sns.scatterplot(
                income_class.sort_values(by='gdp_per_capita',ascending=False),
                y='life_exp',
                x='total_daily_protein',hue='entity',
                palette='copper_r',marker="o",s=150,alpha=.5)
plt.xlabel('Total Daily Protein Intake')
plt.ylabel('Life Expectancy')


plt.legend(loc='best')
plt.ylabel=('')
plt.savefig('GDP prooo.png', transparent=True)
;

# What is the diet of countries with above average life expectancy and below average gdp? 

In [ ]:
low_gdp_high_life =df.loc[(df['gdp_per_capita']< df['gdp_per_capita'].mean()) & (df['life_exp'] > df['life_exp'].mean())]

In [ ]:
low_gdp_high_life_grouped = low_gdp_high_life.groupby('entity').agg({col: 'mean' for col in low_gdp_high_life.columns[3:14]})

In [ ]:
low_gdp_high_life_grouped



# What is the diet of countries with above average gdp and below average life_exp? 

In [ ]:
df.gdp_per_capita.mean()
print(f' The average GDP Per Capita is {df.gdp_per_capita.mean().round()}')

In [ ]:
low_life_high_gdp=df.loc[(df['gdp_per_capita']>= df['gdp_per_capita'].mean()) & (df['life_exp'] < df['life_exp'].mean())]

In [ ]:
low_life_high_gdp_grouped =low_life_high_gdp.groupby('entity').agg({col: 'mean' for col in low_gdp_high_life.columns[3:14]})

In [ ]:
low_life_high_gdp_grouped

# Who's life expectancy above average and what do they eat?

In [ ]:
longest_living=df.loc[df['life_exp']>= 80]
longest_life = longest_living.groupby('entity').agg({'life_exp':'mean'}).sort_values(by='life_exp')
longest_life = longest_life.reset_index()

long_life_list = longest_life.entity.tolist() 

longest_life_countries = df.loc[df['entity'].isin(long_life_list)]
longlife_grouped = longest_life_countries.groupby('entity').agg({'fish':'mean',
                                                                 'poultry':'mean',
                                                                 'pork':'mean',
                                                                 'beef':'mean',
                                                                 'sheep/goat':'mean',
                                                                 'eggs':'mean',
                                                                 'milk':'mean',
                                                                 'daily_cal_animal_protein':'mean',
                                                                 'daily_cal_veg_protein':'mean',
                                                                 'life_exp':'max',
                                                                 'gdp_per_capita':'mean'})


longlife_grouped

# Searching for outliers 

In [ ]:
# here we see high outliers who's protein consumption from vegetables is more than animal protein.

In [ ]:
sns.boxplot(more_veg[['total_daily_protein','life_exp']],color='tan')

;

In [ ]:
Q1 = more_veg['life_exp'].quantile(0.25)
Q3 = more_veg['life_exp'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR 

veg_bound  = more_veg[more_veg['life_exp'] < lower_bound]
veg_bound  =veg_bound.groupby(['entity','year']).agg({'gdp_per_capita':'mean','life_exp':'max','total_daily_protein':'max'}).sort_values(by='life_exp')

In [ ]:
veg_bound = veg_bound.reset_index()
veg_bound

##### In 1994, Rwanda experienced a devastating genocide that had catastrophic consequences for the population, including a drastic drop in life expectancy. Over the course of approximately 100 days, from April to July 1994, an estimated 800,000 people, primarily from the Tutsi ethnic group, were killed by extremist Hutu militias. The violence was widespread and systematic, targeting men, women, and children.

This horrific event significantly impacted Rwanda's demographics and health indicators. The mass killings, combined with disease, displacement, and the destruction of infrastructure, led to a sharp decline in life expectancy, with some estimates showing it falling to as low as 14 years. This figure reflects the enormous loss of life, particularly among adults, and the fact that the remaining population was disproportionately young due to the high number of deaths. The genocide's aftermath left long-term effects on the country's health, economy, and social structure.

In [ ]:
sns.boxplot(more_meat['life_exp'],color='tan');
;

In [ ]:
lQ1 = more_meat['life_exp'].quantile(0.25)
lQ3 = more_meat['life_exp'].quantile(0.75)
lIQR = lQ3 - lQ1

llower_bound = lQ1 - 1.5 * lIQR
lupper_bound = lQ3 + 1.5 * lIQR 

low_meat_outliers  = more_meat[more_meat['life_exp'] < llower_bound]

In [ ]:
lowmeat= low_meat_outliers.groupby(['entity','region','year']).agg({'gdp_per_capita':'mean','life_exp':'mean','total_daily_protein':'mean'}).sort_values(by='total_daily_protein')
lowmeat = lowmeat.reset_index()
lowmeat.entity = lowmeat.entity.str.title()
lowmeat

In [ ]:
sns.boxplot(df.life_exp)


In [ ]:
sns.boxplot(df.life_exp)

rQ1 = df['life_exp'].quantile(0.25)
rQ3 = df['life_exp'].quantile(0.75)
rIQR = rQ3 - rQ1

rlower_bound = rQ1 - 1.5 * rIQR
rupper_bound = rQ3 + 1.5 * rIQR 

low_life  = df[df['life_exp'] < rlower_bound]
low_life.groupby(['year','region']).agg({'gdp_per_capita':'mean','life_exp':'mean','total_daily_protein':'mean',}).sort_values(by='year')

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.scatterplot(y='gdp_per_capita', x='life_exp', data=df,hue='region',palette='binary_r')
plt.title('Life Expectancy vs. GDP per Capita')
plt.subplot(1, 2, 2)
sns.scatterplot(y='total_daily_protein', x='life_exp', data=df,hue='region',palette='binary_r')
plt.title('Life Expectancy vs. Protein Consumption')
plt.tight_layout()

plt.show()


In [ ]:
# for tableau to show the diets of these countries and a map.